In [39]:
import pandas as pd
import json
import numpy as np

In [46]:

df_activities = pd.read_json("activities.json")
df_npo = pd.read_csv("NPOPlayer.csv", sep=";")

df_selected = df_activities[df_activities['activity'] == "Select mediaID"]

userlist = list(df_selected["user_id"].unique())
mediaIDList = list(df_npo["mediaID"].unique())

userID_index = {}
index_userID = {}
for i, user in enumerate(userlist):
    userID_index[user] = i
    index_userID[i] = user

mediaID_index = {}
index_mediaID = {}
for i, mediaID in enumerate(mediaIDList):
    mediaID_index[mediaID] = i
    index_mediaID[i] = mediaID

rows = (len(userlist))
cols = (len(mediaIDList))
#large_empty=np.full((rows,cols),np.nan)
large_empty=np.zeros([rows,cols], dtype=np.int8)

# Make large interaction matrix
# first itteration is over the rows, so the users!
for i, row in enumerate(large_empty):
    useractivities = df_selected[df_selected["user_id"] == index_userID[i]]
    for j in useractivities.iterrows():
        content_id = (j[1]["content_id"])
        columnindex = mediaID_index[content_id]
        row[columnindex] += 1
        print(i, columnindex)


0 87187
1 90635
2 19996
3 90635
3 19504
4 87185
5 19996
6 87185
7 90635
8 69217
8 9498


In [47]:
large_empty.nbytes

1188117

In [48]:
print(type(large_empty))

<class 'numpy.ndarray'>


In [53]:
def get_user_mean(matrix:np.ndarray)-> dict:
    usermean = {}
    for i in range(len(matrix)):
        usermean[index_userID[i]] = 1/np.mean(matrix[i][matrix[i] != 0])
    return usermean

In [54]:
user_mean_dict = get_user_mean(large_empty)

In [55]:
user_mean_dict

{'chwnpxctrh': 1.0,
 'sovhxymdkm': 1.0,
 'teqocdjxcd': 1.0,
 'omwnrpvyls': 1.0,
 'hejlrzhtyn': 1.0,
 'ahjlmkvzfl': 1.0,
 'whvecjnwuj': 1.0,
 'uygiwhabiy': 1.0,
 'inestfknyv': 1.0}

In [70]:
def get_distance_to_other_users(index_a, row_a):
    distance_dict = {}
    row_a_norm = row_a - user_mean_dict[index_userID[index_a]]  # if not binary
    row_a_norm = row_a  # if binary
    for index_b, row_b in enumerate(large_empty):
        if index_a != index_b:
            row_b_norm = row_b - user_mean_dict[index_userID[index_b]]  # if not binary
            row_b_norm = row_b  # if binary
            #dot = (np.dot(row_a_norm, row_b_norm))
            #length = (np.linalg.norm(row_a_norm) * np.linalg(row_b_norm))
            #print(length)
            distance_dict[index_userID[index_b]] = np.dot(row_a_norm, row_b_norm)/(np.linalg.norm(row_a_norm) * np.linalg.norm(row_b_norm))
    return distance_dict


In [71]:
user_distance_list = {}
for index_a, row_a in enumerate(large_empty):
    user_distance_list[index_mediaID[index_a]] = get_distance_to_other_users(index_a, row_a)


In [72]:
user_distance_list

{'POMS_BV_16059067': {'sovhxymdkm': 0.0,
  'teqocdjxcd': 0.0,
  'omwnrpvyls': 0.0,
  'hejlrzhtyn': 0.0,
  'ahjlmkvzfl': 0.0,
  'whvecjnwuj': 0.0,
  'uygiwhabiy': 0.0,
  'inestfknyv': 0.0},
 'POMS_AT_3975232': {'chwnpxctrh': 0.0,
  'teqocdjxcd': 0.0,
  'omwnrpvyls': 0.7071067811865475,
  'hejlrzhtyn': 0.0,
  'ahjlmkvzfl': 0.0,
  'whvecjnwuj': 0.0,
  'uygiwhabiy': 1.0,
  'inestfknyv': 0.0},
 'KN_1685196': {'chwnpxctrh': 0.0,
  'sovhxymdkm': 0.0,
  'omwnrpvyls': 0.0,
  'hejlrzhtyn': 0.0,
  'ahjlmkvzfl': 1.0,
  'whvecjnwuj': 0.0,
  'uygiwhabiy': 0.0,
  'inestfknyv': 0.0},
 'WO_KN_15405842': {'chwnpxctrh': 0.0,
  'sovhxymdkm': 0.7071067811865475,
  'teqocdjxcd': 0.0,
  'hejlrzhtyn': 0.0,
  'ahjlmkvzfl': 0.0,
  'whvecjnwuj': 0.0,
  'uygiwhabiy': 0.7071067811865475,
  'inestfknyv': 0.0},
 'BNN_101372385': {'chwnpxctrh': 0.0,
  'sovhxymdkm': 0.0,
  'teqocdjxcd': 0.0,
  'omwnrpvyls': 0.0,
  'ahjlmkvzfl': 0.0,
  'whvecjnwuj': 1.0,
  'uygiwhabiy': 0.0,
  'inestfknyv': 0.0},
 'POW_05260999': {'chw

In [ ]:
for user in userlist:
    print(user)